In [41]:
import scipy as sp
import numpy as np
import sympy as sy
import scipy.integrate as spi
from sympy.abc import t,x,y, z
import scipy.optimize as spo
from sympy.utilities.lambdify import lambdify 
import itertools as it
import time

In [78]:
#construct homotopy
def H(t, G, F, gamma):
    
    return [(1 - t)*G[i] + gamma*t*F[i] for i in range(len(G))]

#for degree 3 polynomial, do we need to generalise for more than one variable??
def G(x_array):
    func_listG = [i**3 - 1 for i in x_array]
    return func_listG

def F3(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi*x_array[2]**2, x_array[1]**3 + x_array[2]**2 + 8*x_array[0], x_array[2]**3 \
            + x_array[0]*x_array[1]*x_array[2] + np.pi]

def F2(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi, x_array[1]**3 + 8*x_array[0] + 76]

def F1(x_array):
    return [x_array[0]**3 + np.pi + x_array[0]**2 - x_array[0]]

def gamma_generate():
    real = np.random.rand()
    im = np.sqrt(1- real**2)
    return real + im*1j

def G_roots_find(n):
    l = [1, np.exp(1j*2*np.pi/3), np.exp(1j*2*np.pi*2/3)]
    return [i for i in it.product(l, repeat = n)]

In [87]:
def Homotopy_Continuation(t, x_array, F, N, tolerance = 1e-10, tolerance_zero = 1e-10, ratio_tolerance = 1e-5,\
                          N_ratio_tolerance = 50):
    
    time_start = time.time()
    
    num = 0
    delta_t = 1/N
    n = len(x_array)
    gamma = gamma_generate()
    G_roots = G_roots_find(n)
    
    H_func = H(t, G(x_array), F(x_array), gamma)
    H_diff_x = sy.Matrix([[H_func[i].diff(x_array[j]) for i in range(len(x_array))] for j in range(len(x_array))])

    determinant_H = H_diff_x.det()

    H_diff_t = sy.Matrix([H_func[i].diff(t) for i in range(len(x_array))])
    
    if abs(determinant_H) == 0:
        raise TypeError('The determinant of H is zero!')
        
    det_H = lambdify((t, x_array), determinant_H)
        
    H_diff_x_inv = H_diff_x**-1
    H_diff_inv = H_diff_x_inv*H_diff_t
    
    H_Hdiffprime = H_diff_x_inv*sy.Matrix(H_func)
    
    H_Hdiffprime = lambdify((t, x_array), [H_Hdiffprime[i] for i in range(len(H_Hdiffprime))])
        
    H_prime = lambdify((t, x_array), [H_diff_inv[i] for i in range(len(H_diff_inv))])
    
    Hprime_x = lambdify((x_array,t), [H_diff_x[i] for i in range(len(H_diff_x))])    
    H_func_1d = lambdify((x_array,t), H_func)
    

    x_old_arrays = []
    x_olds = []
    
    for x_old in G_roots:
        x_old_array = []
        num += 1
            
        t_n = 0
        
        while t_n <= 1:

            x_old_array.append(x_old)
            t_old = t_n
            t_n += delta_t
            
            if n == 1:
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[-1][-1]
                
                H_func_1 = lambdify((x,t), H_func_1d([x], t)[0])
                Hprime_x_1 = lambdify((x,t), Hprime_x([x], t)[0])

                #x_old = [spo.newton(H_func_1, sol_x, fprime = Hprime_x_1, args=(t_n, ))]
                
            else:
                if abs(det_H(t_n, x_old)) < tolerance_zero:
                    raise TypeError('The determinant of H is zero!')
                    
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[:,-1]
                
                if abs(det_H(t_n, sol_x)) < tolerance_zero:
                    raise TypeError('The determinant of H is zero!')
                    
            #newton's method
            x_old = sol_x
            ratio = np.full(n, 1)
            N_ratio = 0

            while (ratio_all < ratio_tolerance for ratio_all in ratio) and N_ratio < N_ratio_tolerance:
                x_old_intermediate = x_old - H_Hdiffprime(t_n, x_old)

                ratio = abs(x_old_intermediate - x_old)/x_old
                x_old = x_old_intermediate
                N_ratio += 1
                #x_old = spo.newton(H_func, sol_x, fprime = Hprime_x, args=(t_n, ))
                

        #check root is found
        remainder = list(map(abs, F(x_old))) 

        if all(rem < tolerance for rem in remainder) is True:

            x_old = [x_old[i].real if abs(x_old[i].imag) < tolerance_zero else x_old[i] for i in range(len(x_old))]
            
            print('Root {}, \n{}, \nAccuracy {}!!\n'.format(num, x_old, remainder))
            x_olds.append(x_old)
            x_old_arrays.append(x_old_array)

    time_end = time.time()
    
    print('It took {} s to run!'.format(time_end - time_start))
    return x_olds, x_old_arrays

In [96]:
x_sols, x_sols_arrays = Homotopy_Continuation(t, [x, y, z], F3, N=500, tolerance = 1, N_ratio_tolerance= 50, ratio_tolerance= 1e-5)

C:\Users\ASUS\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


Root 1, 
[(-2.32554724654287+1.0004945300220838j), (-1.0387471840041327+2.5288746563953137j), (0.02070229404609798-0.4517597476882026j)], 
Accuracy [0.001213312538642706, 0.0015005722679203614, 0.0002614164073766101]!!

Root 2, 
[(-2.32554724654287+1.0004945300220838j), (-1.0387471840041327+2.5288746563953137j), (0.02070229404609798-0.4517597476882026j)], 
Accuracy [0.001213312538642706, 0.0015005722679203614, 0.0002614164073766101]!!

Root 3, 
[(-0.9975667932868343+2.373089283703179j), (2.5408451845660123-1.0388811006259346j), (0.016600290650125922+0.44306233191590455j)], 
Accuracy [8.250854369404317e-05, 8.410275928648297e-05, 2.0334111254813866e-05]!!

Root 4, 
[(-2.3255068582298106-1.0005227030779544j), (-1.038791519532869-2.5288841683858427j), (0.02068038869516605+0.45178586685210353j)], 
Accuracy [0.0012117883729623928, 0.0014988350566767783, 0.00026108231142745413]!!

Root 5, 
[(2.3686617386237963+0.9451427324820695j), (1.0500337350709283+2.513608660796639j), (0.0060326243606170

In [ ]:
coeff = [5,0,2,np.pi-6]
np.roots(coeff)

In [ ]:
coeff = [9,10,0,3]
np.roots(coeff)

In [76]:
np.full(1,3)

array([3])